## 4. Modelagem com Machine Learning

Objetivo: prever a porcentagem de acertos (PCT_ACERTO) de estudantes de um determinado estado no ENEM, usando diferentes modelos, com processo de melhoria iterativa.

Nesta etapa buscamos responder às seguintes perguntas de pesquisa, a partir da modelagem preditiva:

- Quais fatores estão mais associados à maior porcentagem de acertos no ENEM?
- Até que ponto conseguimos prever o desempenho (PCT_ACERTO) de um estudante, apenas com informações sociodemográficas e escolares?

#### Importações

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
import seaborn as sns

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


#### 1. Carregamento dos dados e filtragem por estado

In [ ]:
DATA_PATH = "dados_tidy.parquet"

try:
    dados = pd.read_parquet(DATA_PATH)
except FileNotFoundError:
    raise FileNotFoundError(
        f"Arquivo {DATA_PATH} não encontrado. "
        "Verifique se ele está na mesma pasta do notebook ou ajuste o caminho em DATA_PATH."
    )

dados.info()

##### Filtrando por estado de interesse e aplicando amostragem

In [ ]:
TARGET_STATE = "SP"

dados_estado = dados[dados["SG_UF_PROVA"] == TARGET_STATE].copy()

print(f"Total de linhas no estado {TARGET_STATE}: {len(dados_estado):,}")

N_SAMPLE = 100_000

if len(dados_estado) > N_SAMPLE:
    dados_modelagem = dados_estado.sample(
        n=N_SAMPLE, random_state=RANDOM_STATE
    ).reset_index(drop=True)
    print(f"Usando amostra de {len(dados_modelagem):,} linhas para modelagem.")
else:
    dados_modelagem = dados_estado.reset_index(drop=True)
    print("Número de linhas menor que N_SAMPLE, usando todos os dados desse estado.")

#### 2. Definição de target e features

In [ ]:
TARGET = "PCT_ACERTO"

cols_leakage = ["NOTA", "ACERTOS", "TOTAL_QUESTOES"]
cols_para_remover = [c for c in cols_leakage if c in dados_modelagem.columns]

X = dados_modelagem.drop(columns=[TARGET] + cols_para_remover)
y = dados_modelagem[TARGET]

print("Formato de X:", X.shape)
print("Formato de y:", y.shape)

#### 3. Split Treino/Teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=RANDOM_STATE,
)

print("Treino:", X_train.shape, y_train.shape)
print("Teste :", X_test.shape, y_test.shape)

#### 4. Pré-processamento (Feature Engineering básica)

In [ ]:
cols_forcar_categoricas = [
    'TP_FAIXA_ETARIA',
    'TP_COR_RACA',
    'TP_ESCOLA',
    'Q022',
    'Q024',
    'Q025',
    'TP_ESTADO_CIVIL',
    'TP_NACIONALIDADE',
    'TP_ST_CONCLUSAO',
    'TP_LINGUA',
    'TP_STATUS_REDACAO'
]

for col in cols_forcar_categoricas:
    if col in X.columns:
        X[col] = X[col].astype("category")

numeric_features = X.select_dtypes(include=["number"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object", "category"]).columns.tolist()

numeric_transformer = Pipeline(
    steps=[
        ("scaler", StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

#### 5. Função auxiliar para avaliação

In [ ]:
def avaliar_modelo(nome, modelo, X_train, X_test, y_train, y_test, cv=None):
    """
    Treina o modelo, avalia em teste e opcionalmente em cross-validation.
    """
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f"\n=== {nome} ===")
    print(f"RMSE teste: {rmse:,.2f}")
    print(f"MAE  teste: {mae:,.2f}")
    print(f"R²   teste: {r2:,.4f}")

    if cv is not None:
        scores = cross_val_score(
            modelo,
            X_train,
            y_train,
            cv=cv,
            scoring="neg_root_mean_squared_error",
            n_jobs=-1,
        )
        scores_rmse = -scores
        print(f"CV ({cv}-fold) - RMSE médio: {scores_rmse.mean():,.2f} (+/- {scores_rmse.std():,.2f})")

    return {
        "modelo": nome,
        "rmse_teste": rmse,
        "mae_teste": mae,
        "r2_teste": r2,
    }

#### 6. Modelo Baseline – Regressão Linear


In [ ]:
baseline_lr = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", LinearRegression())
    ]
)

resultados = []

res_lr = avaliar_modelo(
    "Regressão Linear (Baseline)",
    baseline_lr,
    X_train,
    X_test,
    y_train,
    y_test,
    cv=3,
)

resultados.append(res_lr)

#### 7. Modelo mais complexo – Random Forest

In [ ]:
rf_model = RandomForestRegressor(
    n_estimators=150,
    max_depth=None,
    n_jobs=-1,
    random_state=RANDOM_STATE,
)

rf_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", rf_model),
    ]
)

res_rf = avaliar_modelo(
    "Random Forest (default ajustado)",
    rf_pipeline,
    X_train,
    X_test,
    y_train,
    y_test,
    cv=3,
)

resultados.append(res_rf)

#### 8. Comparação dos modelos

In [ ]:
df_resultados = pd.DataFrame(resultados)
df_resultados

In [ ]:
plt.figure(figsize=(8, 5))
sns.barplot(
    data=df_resultados,
    x="modelo",
    y="rmse_teste"
)
plt.xticks(rotation=20)
plt.title(f"Comparação de RMSE em Teste – Estado {TARGET_STATE}")
plt.ylabel("RMSE (nota)")
plt.show()

#### 9. Otimização de hiperparâmetros – Random Forest

In [ ]:
param_distributions = {
    "model__n_estimators": [100, 150, 200, 300],
    "model__max_depth": [None, 10, 20, 30],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4],
    "model__max_features": ["sqrt", "log2", 0.5],
}

rf_base = RandomForestRegressor(
    n_jobs=-1,
    random_state=RANDOM_STATE,
)

rf_pipeline_tuning = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", rf_base),
    ]
)

random_search = RandomizedSearchCV(
    rf_pipeline_tuning,
    param_distributions=param_distributions,
    n_iter=8,        
    cv=3,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=1,
)

random_search.fit(X_train, y_train)

print("Melhores parâmetros encontrados:")
print(random_search.best_params_)

best_rf_pipeline = random_search.best_estimator_

res_rf_tunado = avaliar_modelo(
    "Random Forest (tunado)",
    best_rf_pipeline,
    X_train,
    X_test,
    y_train,
    y_test,
    cv=3,
)

resultados.append(res_rf_tunado)

df_resultados = pd.DataFrame(resultados)
df_resultados

#### 10. Importância das features

In [ ]:
best_preprocess = best_rf_pipeline.named_steps["preprocess"]
best_model = best_rf_pipeline.named_steps["model"]

ohe = best_preprocess.named_transformers_["cat"].named_steps["onehot"]
cat_cols = categorical_features
num_cols = numeric_features

ohe_feature_names = ohe.get_feature_names_out(cat_cols)
all_feature_names = np.concatenate([num_cols, ohe_feature_names])

importances = best_model.feature_importances_

feat_importances = pd.DataFrame({
    "feature": all_feature_names,
    "importance": importances,
}).sort_values("importance", ascending=False)

feat_importances.head(20)

In [ ]:
plt.figure(figsize=(8, 10))
sns.barplot(
    data=feat_importances.head(50),
    x="importance",
    y="feature"
)
plt.title("Top 20 Features mais importantes – Random Forest tunado")
plt.tight_layout()
plt.show()

Os resultados de importância de features da Random Forest tunada indicam que algumas variáveis se destacam fortemente na explicação do PCT_ACERTO. Entre as principais, aparecem:

- **NU_NOTA_REDACAO** – maior importância individual no modelo. Como a redação é um componente central da prova, faz sentido que a nota de redação esteja altamente associada à porcentagem global de acertos, refletindo habilidades de leitura, argumentação e domínio da língua.
- **AREA_CONHECIMENTO_CH, AREA_CONHECIMENTO_LC, AREA_CONHECIMENTO_MT, AREA_CONHECIMENTO_CN** – indicam a área de conhecimento predominante ou agrupamentos de desempenho, capturando perfis de estudantes que se saem melhor em determinados conjuntos de disciplinas.
- **PIB_MUNICIPIO** e **CO_MUNICIPIO_PROVA** – Contexto socioeconômico e territorial. Municípios com maior PIB tendem a oferecer mais infraestrutura e oportunidades educacionais, o que se reflete em melhor desempenho médio dos estudantes.
- **TP_LINGUA** – língua estrangeira escolhida na prova, que pode estar associada a trajetórias escolares específicas ou a determinados perfis de escola.
- Variáveis categóricas relacionadas a **tipo de escola (TP_ESCOLA)**, **cor/raça (TP_COR_RACA)** e questões do questionário socioeconômico (como **Q001**, **Q002**, **Q024**), que refletem o contexto familiar, nível de escolaridade dos pais e condições de estudo.

#### 11. Resultados

In [ ]:
y_pred_best = best_rf_pipeline.predict(X_test)

residuos = y_test - y_pred_best

plt.figure(figsize=(6, 4))
sns.histplot(residuos, kde=True)
plt.title("Distribuição dos resíduos – melhor modelo")
plt.xlabel("Erro (y_real - y_pred)")
plt.ylabel("Frequência")
plt.show()

plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred_best, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--")
plt.xlabel("PCT_ACERTO real")
plt.ylabel("PCT_ACERTO previsto")
plt.title("Real vs Previsto – melhor modelo")
plt.tight_layout()
plt.show()

#### 12. Trade-off entre complexidade e interpretabilidade

Os resultados mostram um trade-off claro entre modelos simples e complexos:

- A **Regressão Linear (baseline)** oferece alta interpretabilidade: cada coeficiente pode ser entendido como o efeito médio de uma variável sobre o PCT_ACERTO, controladas as demais. Em contrapartida, seu desempenho é um pouco inferior ao melhor modelo.
- A **Random Forest tunada** é mais complexa e menos transparente, mas atinge melhor desempenho. Ela captura relações não lineares e interações que o modelo linear não consegue representar.

Na prática:

- Para **explicar** de forma clara o efeito das variáveis, a Regressão Linear e a análise de importâncias de features da Random Forest são mais adequadas.
- Para **prever** o PCT_ACERTO com o menor erro possível, a Random Forest tunada é a melhor escolha entre os modelos testados.

#### 13. Limitações e possíveis falhas dos modelos

Apesar dos resultados positivos, é importante reconhecer limitações importantes:

- **Escopo geográfico**: os modelos foram treinados apenas com dados de um estado escolhido.
- **Grupos específicos**: a análise de erros sugere que o modelo pode ser menos preciso em faixas extremas de desempenho, o que pode afetar, por exemplo, estudantes com desempenho muito baixo ou muito alto.